# FESD - Feature Estimation for Skeleton Detection

This notebook contains the code for the data analysis and training and testing part of the fault estimation for skeleton detection. The data was recorded using the FESD project in this directory.

---
## Import dependencies

In [1]:
!pip install -r requirements.txt

  Using cached contourpy-1.0.6-cp39-cp39-win_amd64.whl (161 kB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached fonttools-4.38.0-py3-none-any.whl (965 kB)
  Using cached kiwisolver-1.4.4-cp39-cp39-win_amd64.whl (55 kB)
  Using cached matplotlib-3.6.2-cp39-cp39-win_amd64.whl (7.2 MB)
  Using cached opencv_python_headless-4.6.0.66-cp36-abi3-win_amd64.whl (35.5 MB)
  Using cached openni-2.3.0.tar.gz (37 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached packaging-22.0-py3-none-any.whl (42 kB)
  Using cached pandas-1.5.2-cp39-cp39-win_amd64.whl (10.9 MB)
  Using cached Pillow-9.3.0-cp39-cp39-win_amd64.whl (2.5 MB)
  Using cached pyrealsense2-2.53.1.4623-cp39-cp39-win_amd64.whl (12.9 MB)
  Using cached pytz-2022.7-py2.py3-none-any.whl (499 kB)
  Using cached seaborn-0.12.1-py3-none-any.whl (288 kB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
Using legacy 'setup.py install' for openni, sinc

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Users\pohly\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import json
import cv2
import pyrealsense2 as rs
from openni import openni2

sns.set()

---
## Collect recording sessions

In [8]:
rec_path = 'H:\Recordings'
sessions = []

for rec in os.listdir(rec_path):
    if rec.endswith('.json'):
        sessions.append(rec)
        print(rec[:-5])

Session_2022-12-19T10.47.11
Session_2022-12-19T11.03.37
Session_2022-12-20T10.32.54
Session_2022-12-20T11.02.31
Session_2022-12-20T21.33.42
Session_2022-12-20T23.02.58
Session_2022-12-21T17.32.24
Session_2022-12-21T17.36.30
Session_2022-12-26T10.55.29
Session_2022-12-26T11.14.48


In [15]:
rs_sessions = []
orbbec_sessions = []

for session in sessions:
    with open(os.path.join(rec_path, session)) as f:
        data = json.load(f)
        for cam in data['Cameras']:
          if cam['Type'] == 'Realsense':
            rs_sessions.append(cam['FileName'])
          elif cam['Type'] == 'Orbbec':
            orbbec_sessions.append(cam['FileName'])

In [19]:
pipe = rs.pipeline()
cfg = rs.config().enable_device_from_file(os.path.join(rec_path, rs_sessions[2]))
pipe.start(cfg)
device = pipe.get_active_profile().get_device()

frame = pipe.try_wait_for_frames(1000)
depth = frame.get_depth_frame()

depth_profile = rs.video_stream_profile(depth.profile)
intrinsics = depth_profile.get_intrinsics()

print(intrinsics)

RuntimeError: try_wait_for_frames cannot be called if a callback was provided